# Documentation - Links from ODS

### API Overview
https://digital.nhs.uk/services/organisation-data-service/apis-for-the-organisation-data-service#section-4

### Search endpoint
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/search-endpoint

Search parameters
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/search-parameters#name-substring-search-

### Ogranisation endpoint 
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/organisation-endpoint

Org succession explanation
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/how-to-understand-organisation-succession

More endpoints
https://digital.nhs.uk/services/organisation-data-service/guidance-for-developers/ods-ord-api-endpoints

Implementation guide and links
https://directory.spineservices.nhs.uk/ODSAPISuite

In [1]:
import pandas as pd
import requests
import json

In [2]:
#import ods codes csv file
ODSdata = pd.read_csv('Ods codes list.csv')
#select column that contains the ODS codes
ODScodes = ODSdata['SiteODSCode'].tolist()

#alternatively input codes list
ODScodes = ["X26", "T1430", "X09"]

In [4]:
#create empty dataframe
df_succ = pd.DataFrame(columns = ["SiteODScode", "SuccessorODScode", "PredecessorODScode", "Date"])
#populate dataframe
for code in ODScodes:
    response = requests.get("https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations/"+str(code)+"?")
    data = response.json()
    try:
        for suc in data["Organisation"]["Succs"]["Succ"]:
            site = str(data["Organisation"]["OrgId"]["extension"])
            typesp= (suc["Type"])
            if typesp == "Predecessor":
                successor = "-"
                predecessor = (suc["Target"]["OrgId"]["extension"])
            elif typesp == "Successor":
                successor = (suc["Target"]["OrgId"]["extension"])
                predecessor = "-"
            else:
                successor = "-"
                predecessor = "-"

            date = (suc["Date"][0]["Start"])
        
    except:
        site = str(code) + " (Not found)"
        successor = "-"
        predecessor = "-"
        date = "-"

    df_succ = df_succ.append({'SiteODScode' : site , 
                            'SuccessorODScode' : successor, 
                            'PredecessorODScode' :predecessor, 
                            'Date' : date}, ignore_index=True)

#view dataframe
df_succ

,SiteODScode,SuccessorODScode,PredecessorODScode,Date
0,X26,-,X09,2013-04-01
1,T1430,X26,-,2013-04-01
2,X09,X26,-,2013-04-01


In [ ]:
#create output file
df_succ.to_csv("Successors.csv", index = False)